In [1]:
import EEGAnalysis as ea
import h5py
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import re, os
from tqdm import tqdm

In [2]:
dataset = ea.DataManager('/home/yizhan/storage/EEG/Data')

patient_id = 'zhouchen'
patient = dataset.get_patient(patient_id)

In [3]:
_freq = 2000
ROI = (-3, 3)
frange = np.logspace(np.log10(10), np.log10(150), 40)
# frange = np.logspace(np.log10(1), np.log10(20), 30)
zbase = (-2,-1)
tspec = np.linspace(ROI[0], ROI[1], int(_freq)*(ROI[1]-ROI[0]))

In [4]:
nyq = _freq / 2.0
width = 5.0 / nyq
ripple_db = 60.0

# Compute the order and Kaiser parameter for the FIR filter.
N, beta = signal.kaiserord(ripple_db, width)

taps = signal.firwin(N, [50/nyq, 80/nyq],window=('kaiser', beta),pass_zero=False)

# _filted = signal.filtfilt(taps, 1, _data)
# _hilbert_pwr = np.abs(signal.hilbert(_filted))

In [6]:
data_list = patient.load_isplit(110)

sample = data_list['180901-1']

In [8]:
list(data_list.keys())

['180827-1',
 '180827-2',
 '180827-3',
 '180829-1',
 '180829-2',
 '180829-3',
 '180831-1',
 '180831-2',
 '180831-3',
 '180901-1',
 '180901-2',
 '180901-3',
 '180902-1',
 '180902-2',
 '180902-3',
 '180903-1',
 '180903-2',
 '180903-3',
 '180904-1',
 '180904-2',
 '180904-3']

In [12]:
_markers = patient.get_marker('180827-3', paradigm='3-10')
_markers

array([146.2, 156.2, 166.2, 176.2, 186.2, 196.2, 206.2, 216.2, 226.2,
       236.2, 246.2, 256.2, 266.2, 276.2, 286.2, 296.2, 306.2, 316.2,
       326.2, 336.2])

In [7]:
tn = 2000
chn = 110
pbar = tqdm(total = chn * len(list(data_list.keys())))

_result = []

for each in data_list.keys():
#     _markers = patient.get_marker(each)
#     if np.size(_markers) == 0:
#         pbar.update(chn * tn)
#         continue
#     a = np.where(np.diff(_markers) > 15)[0]
#     _head, _tail = _markers[a + np.array([0,1])]
    _param = each[-1]
    _head = patient.get_marker(each, paradigm=_param+'-10')[-1]
    _tail = patient.get_marker(each, paradigm=_param+'-5')[0]
    
    _head, _tail = (_head, tail) if (_head < _tail) else (_tail, _head)
    
    _day_result = []
    for chidx in range(chn):
        _data = patient.load_isplit(chidx, name=each)[each]['value']
        
        _filted = signal.filtfilt(taps, 1, _data)
        _hilbert_pwr = np.abs(signal.hilbert(_filted))
        # _hilbert_pwr
        _cut_entry = _hilbert_pwr[int(_head*_freq):int(_tail*_freq)]
        _cut = np.hstack((np.flipud(_cut_entry), _cut_entry, np.flipud(_cut_entry)))

        _random_marker = np.random.uniform(size=(tn,), high=np.size(_cut)/_freq-3, low=3)

        _count = []
        for _marker in _random_marker:
            _pwr_epoch = ea.create_1d_epoch_bymarker(_cut, [_marker], ROI, _freq)

            _mu = np.reshape(np.mean(_pwr_epoch, 1), (1, 1))
            _sig = np.reshape(np.std(_pwr_epoch, 1), (1, 1))

            _zscore = (_pwr_epoch - _mu) / _sig

            _smooth = ea.decomposition.gaussianwind(np.mean(_zscore, 0), fs=_freq, sigma=0.05)

            _entrain = np.max(_smooth[(tspec > -1) & (tspec < 1)]) > 1.96 and np.max(_smooth[(tspec < -1) | (tspec > 1)]) < 1.96
            _count.append(1 if _entrain else 0)
            
        _day_result.append(np.sum(_count) / np.size(_count))
        pbar.update(1)
    _result.append(_day_result)
pbar.close()


  1%|          | 3/330 [01:15<2:17:44, 25.27s/it]

KeyboardInterrupt: 

In [ ]:
tn = 2000

ch_p = []
pbar = tqdm(total=73 * 2000)
for chidx in range(73):

    _unit, _data, _ = patient.load_isplit(chidx = _chidx, name = _fname)[_fname].values()
    _filted = signal.filtfilt(taps, 1, _data)
    _hilbert_pwr = np.abs(signal.hilbert(_filted))

    _random_marker = np.random.uniform(size=(tn,), high=380, low=280)
    # plt.hist(_random_marker)
    # plt.show()

    _count = []
    for _marker in _random_marker:
        _pwr_epoch = ea.create_1d_epoch_bymarker(_hilbert_pwr, [_marker], ROI, _freq)

        _mu = np.reshape(np.mean(_pwr_epoch, 1), (1, 1))
        _sig = np.reshape(np.std(_pwr_epoch, 1), (1, 1))

        _zscore = (_pwr_epoch - _mu) / _sig

        _smooth = ea.decomposition.gaussianwind(np.mean(_zscore, 0), fs=_freq, sigma=0.05)

        _entrain = np.max(_smooth[(tspec > -1) & (tspec < 1)]) > 1.96 and np.max(_smooth[(tspec < -1) | (tspec > 1)]) < 1.96
        _count.append(1 if _entrain else 0)
        pbar.update(1)
    #     plt.plot(tspec, _smooth)
    #     plt.vlines([tspec[np.argmax(_smooth)]], np.min(_smooth), np.max(_smooth)*1.1)
    #     plt.title('peak time = %.3f sec'%tspec[np.argmax(_smooth)])

    #     plt.show()
    ch_p.append(np.sum(_count) / 2000)
pbar.close()